# Twitter Tweets Analysis as positive or negative

Getting necessary libraries

In [39]:
import numpy as np
import pandas as pd
import nltk 
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
from nltk import pos_tag
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier

In [22]:
testing_data = pd.read_csv('0000000000002747_test_twitter_x_test (1).csv')
training_data= pd.read_csv('0000000000002747_training_twitter_x_y_train (1).csv')

In [23]:
print(training_data.columns)
print(testing_data.columns)

Index(['tweet_id', 'airline_sentiment', 'airline', 'airline_sentiment_gold',
       'name', 'negativereason_gold', 'retweet_count', 'text', 'tweet_coord',
       'tweet_created', 'tweet_location', 'user_timezone'],
      dtype='object')
Index(['tweet_id', 'airline', 'airline_sentiment_gold', 'name',
       'negativereason_gold', 'retweet_count', 'text', 'tweet_coord',
       'tweet_created', 'tweet_location', 'user_timezone'],
      dtype='object')


In [24]:
training_result = training_data['airline_sentiment']

In [25]:
training_data[:10]

,tweet_id,airline_sentiment,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,567900433542488064,negative,Southwest,NaN,ColeyGirouard,NaN,0,"@SouthwestAir I am scheduled for the morning, ...",NaN,2015-02-17 20:16:29 -0800,Washington D.C.,Atlantic Time (Canada)
1,569989168903819264,positive,Southwest,NaN,WalterFaddoul,NaN,0,@SouthwestAir seeing your workers time in and ...,NaN,2015-02-23 14:36:22 -0800,"Indianapolis, Indiana; USA",Central Time (US & Canada)
2,568089179520954368,positive,United,NaN,LocalKyle,NaN,0,@united Flew ORD to Miami and back and had gr...,NaN,2015-02-18 08:46:29 -0800,Illinois,Central Time (US & Canada)
3,568928195581513728,negative,Southwest,NaN,amccarthy19,NaN,0,@SouthwestAir @dultch97 that's horse radish 😤🐴,NaN,2015-02-20 16:20:26 -0800,NaN,Atlantic Time (Canada)
4,568594180014014464,negative,United,NaN,J_Okayy,NaN,0,@united so our flight into ORD was delayed bec...,NaN,2015-02-19 18:13:11 -0800,NaN,Eastern Time (US & Canada)
5,569677636613439488,negative,United,NaN,bobgiolito,NaN,0,@united Why did you load us in this flying sar...,NaN,2015-02-22 17:58:27 -0800,"Los Angeles, CA",Pacific Time (US & Canada)
6,569658903044218880,negative,Delta,NaN,aaronkinnari,NaN,0,@JetBlue that is a stock response. Delays not ...,NaN,2015-02-22 16:44:00 -0800,Gotham,Quito
7,568542766860541952,positive,Delta,NaN,TimothySays,NaN,0,@JetBlue That'd be nice! Hoping to rack up eno...,NaN,2015-02-19 14:48:53 -0800,"Burlington, MA",Eastern Time (US & Canada)
8,570116209263427584,negative,United,NaN,lindaSWC,NaN,1,@united frankly worse customer service ever. P...,NaN,2015-02-23 23:01:11 -0800,NaN,NaN
9,568870144891600896,positive,Southwest,NaN,amyums,NaN,0,@SouthwestAir yeah haha. Never been in one. It...,NaN,2015-02-20 12:29:46 -0800,NaN,Central Time (US & Canada)


Removing unwanted features

In [26]:
def updated_data(data):
    del data['tweet_id']
    del data['name']
    del data['tweet_created']
    del data['tweet_location']
    del data['user_timezone']
    del data['tweet_coord']
    del data['airline_sentiment']
    del data['airline']
    del data['negativereason_gold']
    del data['retweet_count']
    del data['airline_sentiment_gold']
    return data

def updated_data1(data):
    del data['tweet_id']
    del data['name']
    del data['tweet_created']
    del data['tweet_location']
    del data['user_timezone']
    del data['tweet_coord']
    del data['airline']
    del data['negativereason_gold']
    del data['retweet_count']
    del data['airline_sentiment_gold']
    return data

In [27]:
training_data = updated_data(training_data)
testing_data = updated_data1(testing_data)

In [28]:
testing_data[:10]

,text
0,@AmericanAir In car gng to DFW. Pulled over 1h...
1,"@AmericanAir after all, the plane didn’t land ..."
2,@SouthwestAir can't believe how many paying cu...
3,@USAirways I can legitimately say that I would...
4,@AmericanAir still no response from AA. great ...
5,@united we have developers flying down tmrw mo...
6,@USAirways hello??? Anyone there?
7,@USAirways @husainhaqqani Mr. Husain u shld pr...
8,"@USAirways not likely, flightaware says plane ..."
9,@AmericanAir they don't even give an option to...


Getting Stop words and punctuations

In [29]:
lemmatizer = WordNetLemmatizer()
stops = set(stopwords.words('english'))
punctuations = list(string.punctuation)
stops.update(punctuations)

In [30]:
def getSimpleTag(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('R'):
        return wordnet.ADV
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [31]:
def cleanWords(words):
    words = word_tokenize(words)
    output_words = []
    for w in words:
        if w.lower() not in stops:
            #w1 = lemmatizer.lemmatize(w.lower())
            pos = pos_tag([w])
            clean_word = lemmatizer.lemmatize(w, pos = getSimpleTag(pos[0][1]))
            output_words.append(clean_word)      
    return output_words

In [32]:
i = 0
for doc in training_data['text']:
    print(doc)
    if i == 2:
        break
    i+=1

@SouthwestAir I am scheduled for the morning, 2 days after the fact, yes..not sure why my evening flight was the only one Cancelled Flightled
@SouthwestAir seeing your workers time in and time out going above and beyond is why I love flying with you guys. Thank you!
@united Flew ORD to Miami and back and  had great crew, service on both legs. THANKS


In [33]:
words_arr = [cleanWords(doc) for doc in training_data['text']]

Using pre computed saved data

In [34]:
import pickle
pickle_out = open('words_arr.txt', 'wb')
pickle.dump(words_arr, pickle_out)
pickle_out.close()

In [35]:
import pickle
pickle_in = open('words_arr.txt', 'rb')
words_arr = pickle.load(pickle_in)

In [36]:
words_arr[:11]

[['SouthwestAir',
  'schedule',
  'morning',
  '2',
  'days',
  'fact',
  'yes..not',
  'sure',
  'even',
  'flight',
  'one',
  'Cancelled',
  'Flightled'],
 ['SouthwestAir',
  'see',
  'workers',
  'time',
  'time',
  'go',
  'beyond',
  'love',
  'fly',
  'guys',
  'Thank'],
 ['united',
  'Flew',
  'ORD',
  'Miami',
  'back',
  'great',
  'crew',
  'service',
  'legs',
  'THANKS'],
 ['SouthwestAir', 'dultch97', "'s", 'horse', 'radish', '😤🐴'],
 ['united',
  'flight',
  'ORD',
  'delayed',
  'Air',
  'Force',
  'One',
  'last',
  'flight',
  'SBN',
  '8:20',
  '5',
  'mins',
  'land'],
 ['united',
  'load',
  'u',
  'fly',
  'sardine',
  'knew',
  'pilots',
  '2',
  'hours',
  'Late',
  'Flight',
  'incompetent',
  'beyond',
  'belief'],
 ['JetBlue',
  'stock',
  'response',
  'Delays',
  'frustrate',
  'poor',
  'cust',
  'serv',
  'amp',
  'told',
  '3',
  'ppl',
  'wait',
  'amp',
  "'d",
  'come',
  'back'],
 ['JetBlue',
  "'d",
  'nice',
  'Hoping',
  'rack',
  'enough',
  'miles

In [41]:
stemmed_words = []
i = 0
for doc in words_arr:
    stemmed_words.append([])
    for w in doc:
        stemmed_words[i].append(ps.stem(w))
    i += 1

In [42]:
words_arr[:10]

[['SouthwestAir',
  'schedule',
  'morning',
  '2',
  'days',
  'fact',
  'yes..not',
  'sure',
  'even',
  'flight',
  'one',
  'Cancelled',
  'Flightled'],
 ['SouthwestAir',
  'see',
  'workers',
  'time',
  'time',
  'go',
  'beyond',
  'love',
  'fly',
  'guys',
  'Thank'],
 ['united',
  'Flew',
  'ORD',
  'Miami',
  'back',
  'great',
  'crew',
  'service',
  'legs',
  'THANKS'],
 ['SouthwestAir', 'dultch97', "'s", 'horse', 'radish', '😤🐴'],
 ['united',
  'flight',
  'ORD',
  'delayed',
  'Air',
  'Force',
  'One',
  'last',
  'flight',
  'SBN',
  '8:20',
  '5',
  'mins',
  'land'],
 ['united',
  'load',
  'u',
  'fly',
  'sardine',
  'knew',
  'pilots',
  '2',
  'hours',
  'Late',
  'Flight',
  'incompetent',
  'beyond',
  'belief'],
 ['JetBlue',
  'stock',
  'response',
  'Delays',
  'frustrate',
  'poor',
  'cust',
  'serv',
  'amp',
  'told',
  '3',
  'ppl',
  'wait',
  'amp',
  "'d",
  'come',
  'back'],
 ['JetBlue',
  "'d",
  'nice',
  'Hoping',
  'rack',
  'enough',
  'miles

In [43]:
all_words = []
for doc in words_arr:
    all_words += doc

In [44]:
freq = nltk.FreqDist(all_words)
common = freq.most_common(1000)
features = [i[0] for i in common]

In [45]:
count_vec = CountVectorizer(max_features = 3000, ngram_range = (1, 2))
train_data = count_vec.fit_transform(all_words)

In [46]:
count_vec.get_feature_names()[:10]

['00', '000', '00pm', '02', '03', '05', '10', '10 000', '10 24', '10 30']

In [47]:
train_data = count_vec.transform(training_data['text'])

In [48]:
test_data = count_vec.transform(testing_data['text'])

In [49]:
train_data

<10980x3000 sparse matrix of type '<class 'numpy.int64'>'
	with 132295 stored elements in Compressed Sparse Row format>

In [50]:
test_data

<3660x3000 sparse matrix of type '<class 'numpy.int64'>'
	with 43572 stored elements in Compressed Sparse Row format>

In [51]:
training_result.shape, train_data.shape

((10980,), (10980, 3000))

Applying different Classifiers

Training on SVM Classifier

In [81]:
from sklearn.svm import SVC

In [82]:
svc = SVC()
svc.fit(train_data, training_result)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

getting score from svm classifier

In [83]:
svc.score(train_data, training_result)

0.6252276867030966

In [84]:
y_pred_svc = svc.predict(test_data)

In [85]:
set(y_pred_svc)

{'negative', 'positive'}

In [87]:
np.savetxt('y_pred_svc.csv', y_pred_svc, fmt = "%s")

Training on Random Forest Classifier

In [103]:
rnd_clf = RandomForestClassifier(criterion = 'gini', n_estimators = 25, n_jobs = 2, verbose = 7)
rnd_clf.fit(train_data, training_result)

building tree 1 of 25building tree 2 of 25

building tree 3 of 25
building tree 4 of 25
building tree 5 of 25
building tree 6 of 25


[Parallel(n_jobs=2)]: Done   4 tasks      | elapsed:    0.5s


building tree 7 of 25
building tree 8 of 25
building tree 9 of 25
building tree 10 of 25
building tree 11 of 25
building tree 12 of 25
building tree 13 of 25
building tree 14 of 25
building tree 15 of 25
building tree 16 of 25
building tree 17 of 25
building tree 18 of 25
building tree 19 of 25
building tree 20 of 25
building tree 21 of 25
building tree 22 of 25
building tree 23 of 25
building tree 24 of 25
building tree 25 of 25


[Parallel(n_jobs=2)]: Done  25 out of  25 | elapsed:    3.5s finished


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=25, n_jobs=2,
            oob_score=False, random_state=None, verbose=7,
            warm_start=False)

getting score from random forest classifier

In [104]:
print(rnd_clf.score(train_data, training_result))

[Parallel(n_jobs=2)]: Done   4 tasks      | elapsed:    0.0s


0.9936247723132969


[Parallel(n_jobs=2)]: Done  25 out of  25 | elapsed:    0.0s finished


In [105]:
y_pred_rnd = rnd_clf.predict(test_data)
print(set(y_pred_rnd))

[Parallel(n_jobs=2)]: Done   4 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done  25 out of  25 | elapsed:    0.0s finished


{'neutral', 'positive', 'negative'}


In [107]:
np.savetxt('y_pred_rnd.csv', y_pred_rnd, fmt = "%s")